In [1]:
import pyterrier as pt
import os
import numpy as np
import pandas as pd
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

if not pt.started():
  pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


# Dense:
## Index

In [2]:
index_t1 = pt.IndexFactory.of("./data/index/index_t1")

query_path_t1 = "data/publish/English/Queries/train.trec"
topics_t1 = pt.io.read_topics(query_path_t1)

qrels_t1 = pt.io.read_qrels("data/publish/French/Qrels/train.txt")


In [ ]:
import pyterrier_colbert.ranking
colbert_factory = pyterrier_colbert.ranking.ColBERTFactory(
    "http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip", None, None)
colbert = colbert_factory.text_scorer(doc_attr='abstract')

In [3]:
print(index_t1.getMetaIndex().getKeys())

['docno']


In [4]:
pt.text.get_text(index_t1, "text")

ValueError: Index from <org.terrier.structures.Index at 0x7f5e14239590 jclass=org/terrier/structures/Index jself=<LocalRef obj=0x6151b18 at 0x7f5dc7cdfdd0>> did not have requested metaindex key text. Keys present in metaindex are ['docno']

In [5]:
from pyterrier_colbert.indexing import ColBERTIndexer

indexer = ColBERTIndexer("./data/models/colbertv2.0", "./data/index/", "index_t1_colBERT")


TypeError: __init__() missing 1 required positional argument: 'chunksize'

In [ ]:
indexer.index(dataset.get_corpus_iter())

## Retrieval

In [ ]:
pt.text.get_text(index_t1, 'abstract')

In [ ]:
colbert_factory = pyterrier_colbert.ranking.ColBERTFactory(
    "http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip", None, None)
colbert = colbert_factory.text_scorer(doc_attr='text')

In [ ]:
br = pt.BatchRetrieve(index_t1) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> colbert
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> ColBERT'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)